# Balance Sheet Display Demo

This notebook demonstrates the balance sheet visualization capabilities in bilancio, including the new deliverable valuation feature.

In [ ]:
# Import required modules
from decimal import Decimal
from bilancio.engines.system import System
from bilancio.domain.agents.bank import Bank
from bilancio.domain.agents.central_bank import CentralBank
from bilancio.domain.agents.household import Household
from bilancio.domain.agents.treasury import Treasury
from bilancio.analysis.balances import agent_balance, system_trial_balance
from bilancio.analysis.visualization import (
    display_agent_balance_table,
    display_agent_balance_from_balance,
    display_multiple_agent_balances
)
from bilancio.ops.banking import deposit_cash, withdraw_cash

## 1. Setup System and Agents

In [ ]:
# Initialize the system
system = System()

# Create central bank
central_bank = CentralBank(id="cb", name="Federal Reserve", kind="central_bank")
system.bootstrap_cb(central_bank)

# Create other agents
bank = Bank(id="bank1", name="Community Bank", kind="bank")
hh1 = Household(id="hh1", name="The Smiths", kind="household")
hh2 = Household(id="hh2", name="The Johnsons", kind="household")
treasury = Treasury(id="treasury", name="US Treasury", kind="treasury")

# Add agents
system.add_agents([bank, hh1, hh2, treasury])

print(f"System created with {len(system.state.agents)} agents")

## 2. Create Financial Instruments

In [ ]:
# Central bank provides reserves to bank
system.mint_reserves(to_bank_id="bank1", amount=10000)

# Issue cash to households
system.mint_cash(to_agent_id="hh1", amount=1000)
system.mint_cash(to_agent_id="hh2", amount=1500)

# Bank converts some reserves to cash
system.convert_reserves_to_cash(bank_id="bank1", amount=5000)

# Households deposit cash at bank
deposit_cash(system, customer_id="hh1", bank_id="bank1", amount=800)
deposit_cash(system, customer_id="hh2", bank_id="bank1", amount=1200)

print("Financial instruments created")

## 3. NEW: Create Deliverables with Monetary Values

Demonstrating the new feature that solves the issue of mixing monetary values with physical quantities.

In [ ]:
# Create a VALUED deliverable (lawn service worth $50)
system.create_deliverable(
    issuer_id="hh2",
    holder_id="hh1",
    sku="LAWN_SERVICE",
    quantity=1,
    unit_price=Decimal("50.00")  # $50 per lawn service
)

# Create an UNVALUED deliverable (cookies - no monetary value)
system.create_deliverable(
    issuer_id="hh1",
    holder_id="hh2",
    sku="COOKIES",
    quantity=12  # 12 cookies, but no price set
)

# Create another valued deliverable (groceries)
system.create_deliverable(
    issuer_id="treasury",
    holder_id="hh1",
    sku="GROCERIES",
    quantity=5,
    unit_price=Decimal("10.00")  # $10 per unit
)

print("Deliverables created with mixed valuation")

## 4. Display Single Agent Balance Sheet

In [ ]:
# Display bank's balance sheet
print("Community Bank Balance Sheet:\n")
display_agent_balance_table(system, "bank1", format='simple')

## 5. Examine Deliverable Valuation Details

In [ ]:
# Get household 1 balance to see valued deliverables
hh1_balance = agent_balance(system, "hh1")

print("The Smiths' Balance Analysis:")
print(f"\nFinancial Assets: ${hh1_balance.total_financial_assets:,}")
print(f"Non-Financial Asset Value: ${hh1_balance.total_nonfinancial_value or 0}")
print(f"Total Assets: ${hh1_balance.total_financial_assets + (hh1_balance.total_nonfinancial_value or 0)}")

print("\nNon-Financial Assets Detail:")
for sku, details in hh1_balance.nonfinancial_assets_by_kind.items():
    print(f"  {sku}:")
    print(f"    Quantity: {details['quantity']} units")
    if details['value'] is not None:
        print(f"    Value: ${details['value']} (included in totals)")
    else:
        print(f"    Value: Not set (excluded from totals)")

## 6. Compare Household Balance Sheets

Notice how deliverables now show both quantity (in brackets) and monetary value properly.

In [ ]:
# Compare both households
print("Household Balance Sheets Comparison:\n")
display_multiple_agent_balances(system, ["hh1", "hh2"], format='simple')

print("\nKey Points:")
print("- Valued deliverables show monetary amount (e.g., LAWN_SERVICE: $50)")
print("- Unvalued deliverables show '-' for amount (e.g., COOKIES)")
print("- Only valued items included in TOTAL ASSETS calculation")
print("- This solves the problem of mixing units with monetary values!")

## 7. Update Deliverable Pricing

In [ ]:
# Find a deliverable to update (cookies)
cookie_id = None
for contract_id, contract in system.state.contracts.items():
    if getattr(contract, 'sku', None) == 'COOKIES' and contract.asset_holder_id == 'hh2':
        cookie_id = contract_id
        break

if cookie_id:
    print("Before pricing cookies:")
    hh2_balance = agent_balance(system, "hh2")
    print(f"Total non-financial value: ${hh2_balance.total_nonfinancial_value or 0}")
    
    # Now price the cookies
    system.update_deliverable_price(cookie_id, Decimal("2.50"))  # $2.50 per cookie
    
    print("\nAfter pricing cookies at $2.50 each:")
    hh2_balance = agent_balance(system, "hh2")
    print(f"Total non-financial value: ${hh2_balance.total_nonfinancial_value or 0}")
    print(f"Cookie value: 12 × $2.50 = $30")

## 8. System-Wide Trial Balance

In [ ]:
# Get system trial balance
trial_balance = system_trial_balance(system)

print("System-Wide Trial Balance:")
print("="*50)

print("\nFinancial Instruments:")
print(f"  Total Financial Assets: ${trial_balance.total_financial_assets:,}")
print(f"  Total Financial Liabilities: ${trial_balance.total_financial_liabilities:,}")

print("\nNon-Financial Assets:")
print(f"  Total Valued Deliverables: ${trial_balance.total_nonfinancial_value or 0}")

print("\nDeliverable Details:")
for sku, details in trial_balance.nonfinancial_assets_by_kind.items():
    value_str = f"${details['value']}" if details['value'] is not None else "unvalued"
    print(f"  {sku}: {details['quantity']} units ({value_str})")

# Verify double-entry bookkeeping
if trial_balance.total_financial_assets == trial_balance.total_financial_liabilities:
    print("\n✅ Financial system is in balance")
else:
    print("\n❌ Financial system is NOT in balance!")

## 9. Display All Agent Balance Sheets

In [ ]:
# Display all agents
print("Complete System Overview:\n")
all_agent_ids = list(system.state.agents.keys())
display_multiple_agent_balances(system, all_agent_ids, format='simple')

## Summary

### Key Features Demonstrated:

1. **Deliverable Valuation** - Deliverables can now have optional monetary values via `unit_price`
2. **Proper Separation** - Physical quantities shown separately from monetary values
3. **Mixed Handling** - System handles both valued and unvalued deliverables correctly
4. **Dynamic Pricing** - Prices can be updated after creation with `update_deliverable_price()`
5. **Accurate Totals** - Only valued items included in monetary calculations

This solves GitHub issue #3 by ensuring balance sheets no longer mix monetary values with physical quantities.